In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from scipy.stats import pearsonr, spearmanr
from transformers import AutoTokenizer, EsmModel
import time
import lightning as L
import os
import scipy
import scipy.stats
import sklearn.metrics as skmetrics
import matplotlib.pyplot as plt
import seaborn as sns
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


    
class SequenceTransformer(L.LightningModule):
    def __init__(self, lr=1e-3, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.lr = lr
        
        # project one-hot (20) to d_model dimensions
        self.input_proj = nn.Linear(20, d_model)
        
        # transformer encoder
        encoder_layer = nn.TraansformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=256,
            dropout=0.1,
            batch_first=True,
            activation='gelu'
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # output projection: d_model → 20 (one ddG per possible mutation)
        self.output_proj = nn.Linear(d_model, 20)
        
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        # x: (1, L, 20)
        out = self.input_proj(x)      # (1, L, d_model)
        out = self.transformer(out)   # (1, L, d_model)
        out = self.output_proj(out)   # (1, L, 20)
        return out

    def training_step(self, batch, batch_idx):
        x      = batch['sequence']   # (1, L, 20)
        mask   = batch['mask']       # (1, L, 20)
        target = batch['labels']     # (1, L, 20)
        pred   = self(x)             # (1, L, 20)
        loss   = self.loss_fn(pred[mask==1], target[mask==1])
        self.log('train_loss', loss, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x      = batch['sequence']
        mask   = batch['mask']
        target = batch['labels']
        pred   = self(x)
        loss   = self.loss_fn(pred[mask==1], target[mask==1])
        self.log('val_loss', loss, prog_bar=True, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

In [ ]:
class SequenceData(Dataset):
    def __init__(self, csv_file, label_col="ddG_ML"):
        self.df = pd.read_csv(csv_file, sep=",")
        self.df = self.df[self.df.mut_type!="wt"]
        self.df["mutation_pos"] = self.df["mut_type"].apply(lambda x: int(x[1:-1])-1)
        self.df["mutation_to"] = self.df["mut_type"].apply(lambda x: aa_to_int[x[-1]])
        self.df = self.df.groupby("WT_name").agg(list)
        self.wt_names = self.df.index.values
        self.encoded_seqs = {}
        for wt_name in self.wt_names:
            mut_row = self.df.loc[wt_name]
            seq = mut_row["wt_seq"][0]
            self.encoded_seqs[wt_name] = one_hot_encode(seq)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        wt_name = self.wt_names[idx]
        mut_row = self.df.loc[wt_name]
        sequence_encoding = self.encoded_seqs[wt_name]
        mask = torch.zeros((1, len(sequence_encoding), 20))
        target = torch.zeros((1, len(sequence_encoding), 20))
        positions = torch.tensor(mut_row["mutation_pos"])
        amino_acids = torch.tensor(mut_row["mutation_to"])
        labels = torch.tensor(mut_row[self.label_col])
        for i in range(len(sequence_encoding)):
            mask[0,i,amino_acids[positions==i]] = 1
            target[0,i,amino_acids[positions==i]] = labels[positions==i]
        return {"sequence": sequence_encoding[None,:,:].float(), "mask": mask, "labels": target}

In [3]:
# use notebook 3's dataloader
dataset_train = SequenceData('data/mega_train.csv')
dataset_val   = SequenceData('data/mega_val.csv')
loader_train  = DataLoader(dataset_train, batch_size=1, shuffle=True)
loader_val    = DataLoader(dataset_val,   batch_size=1, shuffle=False)

model   = SequenceTransformer(lr=1e-3, d_model=64, nhead=4, num_layers=2)
trainer = L.Trainer(devices=1, max_epochs=20)
trainer.fit(model, loader_train, loader_val)

NameError: name 'SequenceData' is not defined